# Notebook 9: MCP Gateway Fundamentals

## Best Practices for Agentic AI using MCP

---

**Series:** MCP Server Best Practices - Agentic AI Workflows  
**Notebook:** 9 of 13  
**Level:** Advanced  
**Duration:** ~60 minutes

---

## Learning Objectives

By the end of this notebook, you will:

1. Understand the role of MCP gateways in enterprise architectures
2. Learn gateway design patterns and topologies
3. Build a basic MCP gateway from scratch
4. Implement request routing and load balancing
5. Add authentication and authorization at the gateway level
6. Configure tool filtering and policy enforcement

## 1. Why MCP Gateways?

### The Direct Connection Problem

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    WITHOUT A GATEWAY: DIRECT CONNECTIONS                 │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌─────────┐         ┌─────────────┐                                   │
│   │ Client  │────────►│ MCP Server  │   Problems:                       │
│   │    A    │         │     1       │   • Each client needs all creds   │
│   └─────────┘    ┌───►└─────────────┘   • No centralized auth           │
│        │         │                      • No request logging            │
│        │    ┌────┴────┐                 • No rate limiting              │
│        └───►│ Server 2│                 • No tool filtering             │
│             └─────────┘                 • Hard to scale                 │
│                  │                      • Config in every client        │
│   ┌─────────┐    │                                                      │
│   │ Client  │────┼───►┌─────────────┐                                   │
│   │    B    │    └───►│ MCP Server  │                                   │
│   └─────────┘         │     3       │                                   │
│                       └─────────────┘                                   │
│                                                                          │
│   N clients × M servers = N×M connections to manage!                    │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### The Gateway Solution

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    WITH A GATEWAY: CENTRALIZED CONTROL                   │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌─────────┐                              ┌─────────────┐              │
│   │ Client  │──┐                     ┌────►│ MCP Server  │              │
│   │    A    │  │                     │     │     1       │              │
│   └─────────┘  │                     │     └─────────────┘              │
│                │    ┌─────────────┐  │                                  │
│                ├───►│             │──┼────►┌─────────────┐              │
│                │    │   GATEWAY   │  │     │ MCP Server  │              │
│   ┌─────────┐  │    │             │  │     │     2       │              │
│   │ Client  │──┼───►│ • Auth      │──┤     └─────────────┘              │
│   │    B    │  │    │ • Routing   │  │                                  │
│   └─────────┘  │    │ • Logging   │  │     ┌─────────────┐              │
│                │    │ • Filtering │  └────►│ MCP Server  │              │
│   ┌─────────┐  │    │ • Rate Limit│        │     3       │              │
│   │ Client  │──┘    └─────────────┘        └─────────────┘              │
│   │    C    │                                                           │
│   └─────────┘                                                           │
│                                                                          │
│   Benefits:                                                             │
│   ✅ Single point of authentication                                     │
│   ✅ Centralized logging and audit                                      │
│   ✅ Tool filtering by user/role                                        │
│   ✅ Rate limiting and quotas                                           │
│   ✅ Load balancing across servers                                      │
│   ✅ Protocol translation if needed                                     │
│   ✅ Simplified client configuration                                    │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Setup for this notebook
import json
from typing import Dict, List, Any, Optional, Set
from dataclasses import dataclass, field
from datetime import datetime
from enum import Enum
import hashlib
import asyncio

def pprint(obj: dict, title: str = None):
    if title:
        print(f"\n{'='*60}")
        print(f" {title}")
        print(f"{'='*60}")
    print(json.dumps(obj, indent=2, default=str))

## 2. Gateway Architecture Patterns

### Pattern 1: Simple Proxy Gateway

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    SIMPLE PROXY GATEWAY                                  │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   Client ───► Gateway ───► Single MCP Server                            │
│                                                                          │
│   Features:                                                             │
│   • Pass-through with logging                                           │
│   • Authentication wrapper                                              │
│   • Rate limiting                                                       │
│                                                                          │
│   Use Case: Adding enterprise features to existing MCP server           │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### Pattern 2: Aggregator Gateway

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    AGGREGATOR GATEWAY                                    │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│                              ┌──► MCP Server 1 (tools A, B)             │
│   Client ───► Gateway ──────┼──► MCP Server 2 (tools C, D)             │
│                              └──► MCP Server 3 (tools E, F)             │
│                                                                          │
│   Features:                                                             │
│   • Combines tools from multiple servers                                │
│   • Single tools/list returns all tools                                 │
│   • Routes tool calls to correct backend                                │
│                                                                          │
│   Use Case: Unified interface to multiple specialized servers           │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### Pattern 3: Policy Gateway

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    POLICY GATEWAY                                        │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   Client ───► Gateway ───► Policy Engine ───► MCP Servers               │
│                    │              │                                     │
│                    │              ▼                                     │
│                    │        ┌──────────┐                                │
│                    │        │ Policies │                                │
│                    │        │ • RBAC   │                                │
│                    │        │ • Quotas │                                │
│                    │        │ • Filters│                                │
│                    │        └──────────┘                                │
│                    │                                                    │
│                    └──► Audit Log                                       │
│                                                                          │
│   Features:                                                             │
│   • Role-based tool access                                              │
│   • Input/output validation                                             │
│   • Compliance logging                                                  │
│                                                                          │
│   Use Case: Enterprise with strict security requirements                │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

## 3. Core Gateway Components

Let's build a gateway from its core components:

In [ ]:
# Core Gateway Data Structures

@dataclass
class MCPServerConfig:
    """Configuration for a backend MCP server."""
    name: str
    url: str
    transport: str = "sse"  # "sse" or "stdio"
    enabled: bool = True
    health_check_path: str = "/health"
    timeout_seconds: int = 30
    
    # For STDIO transport
    command: Optional[str] = None
    args: List[str] = field(default_factory=list)
    env: Dict[str, str] = field(default_factory=dict)


@dataclass
class ToolInfo:
    """Information about a tool from a backend server."""
    name: str
    description: str
    schema: Dict[str, Any]
    server_name: str  # Which server provides this tool


@dataclass
class GatewayUser:
    """A user authenticated to the gateway."""
    user_id: str
    username: str
    roles: Set[str] = field(default_factory=set)
    allowed_tools: Optional[Set[str]] = None  # None = all tools
    denied_tools: Set[str] = field(default_factory=set)
    rate_limit: int = 100  # requests per minute


@dataclass
class GatewayRequest:
    """An incoming request to the gateway."""
    request_id: str
    method: str  # JSON-RPC method
    params: Dict[str, Any]
    user: GatewayUser
    timestamp: datetime = field(default_factory=datetime.now)


@dataclass
class GatewayResponse:
    """Response from the gateway."""
    request_id: str
    success: bool
    result: Optional[Any] = None
    error: Optional[Dict[str, Any]] = None
    backend_server: Optional[str] = None
    latency_ms: float = 0


# Demo
server_config = MCPServerConfig(
    name="github-server",
    url="http://localhost:8001/sse",
    transport="sse"
)

user = GatewayUser(
    user_id="user-123",
    username="alice",
    roles={"developer", "admin"},
    denied_tools={"delete_repository"}  # Deny dangerous tools
)

print("Gateway Data Structures:")
print("=" * 60)
print(f"\nServer Config: {server_config}")
print(f"\nUser: {user}")

## 4. Building a Basic MCP Gateway

### Gateway Core Class

In [ ]:
# Basic MCP Gateway Implementation

class MCPGateway:
    """A basic MCP gateway that aggregates multiple backend servers."""
    
    def __init__(self, name: str = "MCP Gateway"):
        self.name = name
        self.servers: Dict[str, MCPServerConfig] = {}
        self.tools: Dict[str, ToolInfo] = {}  # tool_name -> ToolInfo
        self.users: Dict[str, GatewayUser] = {}
        self.request_log: List[Dict[str, Any]] = []
    
    # ==================== Server Management ====================
    
    def register_server(self, config: MCPServerConfig):
        """Register a backend MCP server."""
        self.servers[config.name] = config
        print(f"Registered server: {config.name} at {config.url}")
    
    def discover_tools(self, server_name: str, tools: List[Dict]):
        """Register tools discovered from a backend server."""
        for tool in tools:
            tool_info = ToolInfo(
                name=tool["name"],
                description=tool.get("description", ""),
                schema=tool.get("inputSchema", {}),
                server_name=server_name
            )
            self.tools[tool["name"]] = tool_info
        print(f"Discovered {len(tools)} tools from {server_name}")
    
    # ==================== User Management ====================
    
    def register_user(self, user: GatewayUser):
        """Register a user with the gateway."""
        self.users[user.user_id] = user
    
    def authenticate(self, api_key: str) -> Optional[GatewayUser]:
        """Authenticate a user by API key.
        
        In production, this would validate against a real auth system.
        """
        # Simple demo: hash the key and look up
        user_id = hashlib.sha256(api_key.encode()).hexdigest()[:16]
        return self.users.get(user_id)
    
    # ==================== Authorization ====================
    
    def authorize_tool(self, user: GatewayUser, tool_name: str) -> bool:
        """Check if user is authorized to use a tool."""
        # Check denied list first
        if tool_name in user.denied_tools:
            return False
        
        # If allowed_tools is set, tool must be in it
        if user.allowed_tools is not None:
            return tool_name in user.allowed_tools
        
        # Default: allow
        return True
    
    # ==================== Request Handling ====================
    
    def handle_tools_list(self, user: GatewayUser) -> List[Dict]:
        """Handle tools/list request with user-specific filtering."""
        visible_tools = []
        
        for tool_name, tool_info in self.tools.items():
            if self.authorize_tool(user, tool_name):
                visible_tools.append({
                    "name": tool_info.name,
                    "description": tool_info.description,
                    "inputSchema": tool_info.schema
                })
        
        return visible_tools
    
    def route_tool_call(
        self, 
        tool_name: str, 
        arguments: Dict[str, Any],
        user: GatewayUser
    ) -> GatewayResponse:
        """Route a tool call to the appropriate backend server."""
        request_id = hashlib.sha256(
            f"{tool_name}{datetime.now().isoformat()}".encode()
        ).hexdigest()[:12]
        
        start_time = datetime.now()
        
        # Check authorization
        if not self.authorize_tool(user, tool_name):
            return GatewayResponse(
                request_id=request_id,
                success=False,
                error={
                    "code": -32600,
                    "message": f"User not authorized to use tool: {tool_name}"
                }
            )
        
        # Find the tool
        tool_info = self.tools.get(tool_name)
        if not tool_info:
            return GatewayResponse(
                request_id=request_id,
                success=False,
                error={
                    "code": -32601,
                    "message": f"Tool not found: {tool_name}"
                }
            )
        
        # Get the backend server
        server = self.servers.get(tool_info.server_name)
        if not server or not server.enabled:
            return GatewayResponse(
                request_id=request_id,
                success=False,
                error={
                    "code": -32603,
                    "message": f"Backend server unavailable: {tool_info.server_name}"
                }
            )
        
        # In production: forward request to backend server
        # result = await self.forward_to_backend(server, tool_name, arguments)
        
        # Simulated response
        latency = (datetime.now() - start_time).total_seconds() * 1000
        
        # Log the request
        self.log_request(user, tool_name, arguments, True)
        
        return GatewayResponse(
            request_id=request_id,
            success=True,
            result={"content": [{"type": "text", "text": f"Tool {tool_name} executed"}]},
            backend_server=server.name,
            latency_ms=latency
        )
    
    # ==================== Logging ====================
    
    def log_request(
        self, 
        user: GatewayUser, 
        tool_name: str, 
        arguments: Dict,
        success: bool
    ):
        """Log a request for audit."""
        self.request_log.append({
            "timestamp": datetime.now().isoformat(),
            "user_id": user.user_id,
            "username": user.username,
            "tool": tool_name,
            "arguments_hash": hashlib.sha256(
                json.dumps(arguments, sort_keys=True).encode()
            ).hexdigest()[:16],
            "success": success
        })


# Demo
gateway = MCPGateway("Enterprise Gateway")

# Register backend servers
gateway.register_server(MCPServerConfig(
    name="github-mcp",
    url="http://localhost:8001/sse"
))
gateway.register_server(MCPServerConfig(
    name="database-mcp",
    url="http://localhost:8002/sse"
))

# Simulate tool discovery
gateway.discover_tools("github-mcp", [
    {"name": "create_issue", "description": "Create GitHub issue"},
    {"name": "list_repos", "description": "List repositories"},
    {"name": "delete_repository", "description": "Delete a repository"}
])
gateway.discover_tools("database-mcp", [
    {"name": "query", "description": "Execute SQL query"},
    {"name": "insert", "description": "Insert data"}
])

# Register users
developer = GatewayUser(
    user_id="dev-001",
    username="alice",
    roles={"developer"},
    denied_tools={"delete_repository"}  # Can't delete repos
)
gateway.register_user(developer)

admin = GatewayUser(
    user_id="admin-001",
    username="bob",
    roles={"admin"}
    # No restrictions
)
gateway.register_user(admin)

print("\n" + "=" * 60)
print("Gateway Demo")
print("=" * 60)

# Test tools/list for different users
print(f"\nTools visible to developer ({developer.username}):")
for tool in gateway.handle_tools_list(developer):
    print(f"  - {tool['name']}")

print(f"\nTools visible to admin ({admin.username}):")
for tool in gateway.handle_tools_list(admin):
    print(f"  - {tool['name']}")

# Test tool call authorization
print("\nTool call tests:")
result = gateway.route_tool_call("create_issue", {"title": "Test"}, developer)
print(f"  Developer calling create_issue: {result.success}")

result = gateway.route_tool_call("delete_repository", {"repo": "test"}, developer)
print(f"  Developer calling delete_repository: {result.success} (expected: False)")

## 5. Request Routing Strategies

### Routing Based on Tool Name (Default)

In [ ]:
# Advanced Routing Strategies

class RoutingStrategy(Enum):
    TOOL_NAME = "tool_name"     # Route by tool name
    ROUND_ROBIN = "round_robin" # Distribute evenly
    LEAST_LOAD = "least_load"   # Route to least busy
    STICKY = "sticky"           # Same user → same server


@dataclass
class ServerHealth:
    """Health status of a backend server."""
    server_name: str
    healthy: bool
    last_check: datetime
    response_time_ms: float
    active_connections: int
    error_rate: float  # 0.0 to 1.0


class LoadBalancer:
    """Load balancer for distributing requests across servers."""
    
    def __init__(self, strategy: RoutingStrategy = RoutingStrategy.LEAST_LOAD):
        self.strategy = strategy
        self.server_health: Dict[str, ServerHealth] = {}
        self.round_robin_index: int = 0
        self.user_affinity: Dict[str, str] = {}  # user_id -> server_name
    
    def update_health(self, health: ServerHealth):
        """Update health status for a server."""
        self.server_health[health.server_name] = health
    
    def get_healthy_servers(self, candidates: List[str]) -> List[str]:
        """Filter to only healthy servers."""
        return [
            name for name in candidates
            if name in self.server_health 
            and self.server_health[name].healthy
            and self.server_health[name].error_rate < 0.5
        ]
    
    def select_server(
        self, 
        candidates: List[str], 
        user_id: Optional[str] = None
    ) -> Optional[str]:
        """Select a server based on the routing strategy."""
        healthy = self.get_healthy_servers(candidates)
        if not healthy:
            return None
        
        if self.strategy == RoutingStrategy.ROUND_ROBIN:
            server = healthy[self.round_robin_index % len(healthy)]
            self.round_robin_index += 1
            return server
        
        elif self.strategy == RoutingStrategy.LEAST_LOAD:
            return min(
                healthy,
                key=lambda s: self.server_health[s].active_connections
            )
        
        elif self.strategy == RoutingStrategy.STICKY:
            if user_id and user_id in self.user_affinity:
                affinity = self.user_affinity[user_id]
                if affinity in healthy:
                    return affinity
            # Assign new affinity
            server = healthy[0]
            if user_id:
                self.user_affinity[user_id] = server
            return server
        
        # Default: first healthy
        return healthy[0]


# Demo
balancer = LoadBalancer(RoutingStrategy.LEAST_LOAD)

# Simulate health data
balancer.update_health(ServerHealth(
    server_name="server-1",
    healthy=True,
    last_check=datetime.now(),
    response_time_ms=50,
    active_connections=10,
    error_rate=0.01
))
balancer.update_health(ServerHealth(
    server_name="server-2",
    healthy=True,
    last_check=datetime.now(),
    response_time_ms=45,
    active_connections=5,
    error_rate=0.02
))
balancer.update_health(ServerHealth(
    server_name="server-3",
    healthy=False,  # Unhealthy!
    last_check=datetime.now(),
    response_time_ms=0,
    active_connections=0,
    error_rate=1.0
))

print("Load Balancer Demo (Least Load Strategy):")
print("=" * 60)

candidates = ["server-1", "server-2", "server-3"]
for i in range(5):
    selected = balancer.select_server(candidates)
    print(f"Request {i + 1} → {selected}")

## 6. Tool Filtering and Policy Enforcement

### Role-Based Tool Access Control (RBAC)

In [ ]:
# Role-Based Access Control for Tools

@dataclass
class ToolPolicy:
    """Policy for controlling access to a tool."""
    tool_name: str
    allowed_roles: Set[str] = field(default_factory=set)
    denied_roles: Set[str] = field(default_factory=set)
    requires_approval: bool = False
    max_calls_per_hour: Optional[int] = None
    audit_level: str = "standard"  # "none", "standard", "detailed"


class PolicyEngine:
    """Engine for evaluating access policies."""
    
    def __init__(self):
        self.tool_policies: Dict[str, ToolPolicy] = {}
        self.default_policy = ToolPolicy(
            tool_name="*",
            audit_level="standard"
        )
        self.call_counts: Dict[str, Dict[str, int]] = {}  # user_id -> {tool: count}
    
    def add_policy(self, policy: ToolPolicy):
        """Add a tool policy."""
        self.tool_policies[policy.tool_name] = policy
    
    def get_policy(self, tool_name: str) -> ToolPolicy:
        """Get policy for a tool, or default."""
        return self.tool_policies.get(tool_name, self.default_policy)
    
    def check_access(
        self, 
        user: GatewayUser, 
        tool_name: str
    ) -> tuple[bool, str]:
        """Check if user can access tool. Returns (allowed, reason)."""
        policy = self.get_policy(tool_name)
        
        # Check denied roles first
        if policy.denied_roles & user.roles:
            return False, f"User role denied access to {tool_name}"
        
        # Check allowed roles (if specified)
        if policy.allowed_roles:
            if not (policy.allowed_roles & user.roles):
                return False, f"User lacks required role for {tool_name}"
        
        # Check rate limit
        if policy.max_calls_per_hour:
            user_calls = self.call_counts.get(user.user_id, {})
            tool_calls = user_calls.get(tool_name, 0)
            if tool_calls >= policy.max_calls_per_hour:
                return False, f"Rate limit exceeded for {tool_name}"
        
        # Check if approval required
        if policy.requires_approval:
            return False, f"Tool {tool_name} requires approval"
        
        return True, "Access granted"
    
    def record_call(self, user_id: str, tool_name: str):
        """Record a tool call for rate limiting."""
        if user_id not in self.call_counts:
            self.call_counts[user_id] = {}
        if tool_name not in self.call_counts[user_id]:
            self.call_counts[user_id][tool_name] = 0
        self.call_counts[user_id][tool_name] += 1


# Demo
policy_engine = PolicyEngine()

# Define policies
policy_engine.add_policy(ToolPolicy(
    tool_name="query",
    allowed_roles={"developer", "analyst", "admin"},
    max_calls_per_hour=100,
    audit_level="detailed"
))

policy_engine.add_policy(ToolPolicy(
    tool_name="delete_data",
    allowed_roles={"admin"},
    denied_roles={"intern"},
    requires_approval=True,
    audit_level="detailed"
))

policy_engine.add_policy(ToolPolicy(
    tool_name="read_file",
    max_calls_per_hour=50,
    audit_level="standard"
))

# Test users
analyst = GatewayUser(
    user_id="analyst-001",
    username="carol",
    roles={"analyst"}
)

intern = GatewayUser(
    user_id="intern-001",
    username="dave",
    roles={"intern"}
)

admin_user = GatewayUser(
    user_id="admin-002",
    username="eve",
    roles={"admin"}
)

print("Policy Engine Demo:")
print("=" * 60)

test_cases = [
    (analyst, "query"),
    (analyst, "delete_data"),
    (intern, "delete_data"),
    (admin_user, "delete_data"),
    (intern, "read_file"),
]

for user, tool in test_cases:
    allowed, reason = policy_engine.check_access(user, tool)
    status = "✅" if allowed else "❌"
    print(f"\n{status} {user.username} → {tool}")
    print(f"   Reason: {reason}")

## 7. Gateway as MCP Server

The gateway itself is an MCP server that clients connect to:

In [ ]:
# Gateway as FastMCP Server
GATEWAY_MCP_SERVER = '''
from fastmcp import FastMCP
from typing import Dict, List, Any
import httpx
import asyncio

# Create gateway MCP server
gateway = FastMCP("MCP-Gateway")

# Backend server registry
BACKENDS = {
    "github": "http://localhost:8001/sse",
    "database": "http://localhost:8002/sse",
    "filesystem": "http://localhost:8003/sse",
}

# Cached tools from backends
TOOL_REGISTRY: Dict[str, Dict[str, Any]] = {}  # tool_name -> {backend, schema}


async def discover_backend_tools():
    """Discover tools from all backend servers."""
    for backend_name, url in BACKENDS.items():
        try:
            # In real implementation: establish MCP connection and call tools/list
            # For demo, we\'ll simulate
            pass
        except Exception as e:
            print(f"Failed to discover tools from {backend_name}: {e}")


@gateway.tool()
async def list_available_backends() -> List[str]:
    """List all available backend MCP servers.
    
    Returns:
        List of backend server names
    """
    return list(BACKENDS.keys())


@gateway.tool()
async def forward_tool_call(
    backend: str,
    tool_name: str,
    arguments: Dict[str, Any]
) -> Dict[str, Any]:
    """Forward a tool call to a specific backend server.
    
    Args:
        backend: Name of the backend server
        tool_name: Name of the tool to call
        arguments: Arguments to pass to the tool
    
    Returns:
        Result from the backend tool
    """
    if backend not in BACKENDS:
        return {"error": f"Unknown backend: {backend}"}
    
    # In real implementation: forward to backend MCP server
    # For demo, return simulated response
    return {
        "backend": backend,
        "tool": tool_name,
        "result": "Simulated result"
    }


# The gateway can also expose aggregated tools directly
# by dynamically registering tools discovered from backends

if __name__ == "__main__":
    # Discover tools on startup
    asyncio.run(discover_backend_tools())
    
    # Run gateway
    gateway.run(transport="sse", host="0.0.0.0", port=8000)
'''

print("Gateway as MCP Server:")
print("=" * 60)
print(GATEWAY_MCP_SERVER)

## 8. Gateway Configuration

### Configuration File Format

In [ ]:
# Gateway Configuration Example
gateway_config = {
    "gateway": {
        "name": "Enterprise MCP Gateway",
        "version": "1.0.0",
        "host": "0.0.0.0",
        "port": 8000,
        "log_level": "INFO"
    },
    
    "authentication": {
        "type": "oauth2",
        "issuer": "https://auth.company.com",
        "audience": "mcp-gateway",
        "jwks_uri": "https://auth.company.com/.well-known/jwks.json"
    },
    
    "backends": [
        {
            "name": "github-mcp",
            "url": "http://github-mcp:8001/sse",
            "transport": "sse",
            "timeout": 30,
            "health_check": "/health",
            "enabled": True
        },
        {
            "name": "database-mcp",
            "url": "http://database-mcp:8002/sse",
            "transport": "sse",
            "timeout": 60,
            "enabled": True
        },
        {
            "name": "local-files",
            "transport": "stdio",
            "command": "python",
            "args": ["-m", "mcp_server_filesystem"],
            "env": {
                "ALLOWED_DIRS": "/data,/reports"
            },
            "enabled": True
        }
    ],
    
    "policies": {
        "default": {
            "rate_limit": 100,
            "rate_limit_window": "1m",
            "audit_level": "standard"
        },
        "tools": {
            "delete_*": {
                "allowed_roles": ["admin"],
                "requires_approval": True,
                "audit_level": "detailed"
            },
            "query": {
                "allowed_roles": ["developer", "analyst", "admin"],
                "rate_limit": 1000
            }
        }
    },
    
    "routing": {
        "strategy": "least_load",
        "health_check_interval": "30s",
        "circuit_breaker": {
            "enabled": True,
            "threshold": 5,
            "timeout": "60s"
        }
    },
    
    "observability": {
        "metrics": {
            "enabled": True,
            "endpoint": "/metrics",
            "format": "prometheus"
        },
        "tracing": {
            "enabled": True,
            "exporter": "otlp",
            "endpoint": "http://jaeger:4317"
        },
        "logging": {
            "format": "json",
            "include_request_body": False,
            "include_response_body": False
        }
    }
}

print("Gateway Configuration Example:")
print("=" * 60)
print(json.dumps(gateway_config, indent=2))

## 9. Client Configuration for Gateway

### Claude Desktop Configuration

In [ ]:
# Claude Desktop config pointing to gateway
claude_config = {
    "mcpServers": {
        "enterprise-gateway": {
            "url": "https://mcp-gateway.company.com/sse",
            "headers": {
                "Authorization": "Bearer ${MCP_GATEWAY_TOKEN}"
            }
        }
    }
}

print("Claude Desktop Configuration (Gateway):")
print("=" * 60)
print(json.dumps(claude_config, indent=2))
print("\n📁 Save to: %APPDATA%\\Claude\\claude_desktop_config.json")
print("\n✅ Benefits of connecting through gateway:")
print("   • Single endpoint for all tools")
print("   • Centralized authentication")
print("   • Tools filtered by your role")
print("   • Usage tracked and audited")

## 10. Key Takeaways

### Gateway Benefits Summary

| Feature | Without Gateway | With Gateway |
|---------|----------------|---------------|
| **Authentication** | Per-server | Centralized |
| **Authorization** | None or custom | RBAC policies |
| **Tool Discovery** | Manual | Automatic aggregation |
| **Logging** | Per-server | Unified audit log |
| **Rate Limiting** | None | Configurable |
| **Load Balancing** | None | Built-in |
| **Client Config** | Complex | Single endpoint |

### When to Use a Gateway

| ✅ Use Gateway For | ❌ Don't Need Gateway For |
|-------------------|-------------------------|
| Multi-user environments | Single developer |
| Enterprise deployments | Local development |
| Compliance requirements | Simple prototypes |
| Multiple MCP servers | Single MCP server |
| Role-based access | Open access |

### Next Notebook

In **Notebook 10: Enterprise MCP Patterns**, we'll explore:
- Multi-tenant gateway architectures
- Compliance and audit requirements
- Real-world patterns from Red Hat and Apigee
- Production deployment strategies

## 11. Exercises

### Exercise 1: Design Gateway Policies

Create policies for a financial services company with these roles:
- Analyst (read-only data access)
- Trader (can execute trades)
- Compliance (can audit everything)
- Admin (full access)

In [ ]:
# Exercise 1: Design your policy configuration

financial_policies = {
    "tools": {
        "get_portfolio": {
            # Your policy here
        },
        "execute_trade": {
            # Your policy here  
        },
        "view_audit_log": {
            # Your policy here
        },
        "modify_user": {
            # Your policy here
        }
    }
}

print("Your Financial Services Policies:")
print(json.dumps(financial_policies, indent=2))

### Exercise 2: Implement Health Check

Add a health check system that:
- Pings each backend every 30 seconds
- Marks servers unhealthy after 3 failures
- Recovers after 2 successful checks

In [ ]:
# Exercise 2: Implement health checking

class HealthChecker:
    """Health checker for backend MCP servers."""
    
    def __init__(self, failure_threshold: int = 3, recovery_threshold: int = 2):
        self.failure_threshold = failure_threshold
        self.recovery_threshold = recovery_threshold
        self.failure_counts: Dict[str, int] = {}
        self.success_counts: Dict[str, int] = {}
        self.server_status: Dict[str, bool] = {}
    
    async def check_server(self, server_name: str, url: str) -> bool:
        """Check if a server is healthy."""
        # Your implementation here
        pass
    
    def update_status(self, server_name: str, is_healthy: bool):
        """Update server status based on health check result."""
        # Your implementation here
        pass
    
    def is_healthy(self, server_name: str) -> bool:
        """Check if a server is currently marked healthy."""
        # Your implementation here
        pass

# Test your health checker
checker = HealthChecker()
# Simulate checks...

### Exercise 3: Build Tool Aggregator

Create a function that:
- Collects tools from multiple backends
- Prefixes tool names with server name to avoid conflicts
- Returns unified tools list

In [ ]:
# Exercise 3: Build tool aggregator

def aggregate_tools(backend_tools: Dict[str, List[Dict]]) -> List[Dict]:
    """Aggregate tools from multiple backends.
    
    Args:
        backend_tools: Dict mapping backend name to list of tools
            Example: {"github": [{"name": "create_issue", ...}]}
    
    Returns:
        Unified list of tools with prefixed names
            Example: [{"name": "github.create_issue", ...}]
    """
    # Your implementation here
    pass

# Test data
test_backends = {
    "github": [
        {"name": "create_issue", "description": "Create issue"},
        {"name": "list_repos", "description": "List repos"}
    ],
    "database": [
        {"name": "query", "description": "Run SQL"},
        {"name": "insert", "description": "Insert data"}
    ]
}

# aggregated = aggregate_tools(test_backends)
# print(json.dumps(aggregated, indent=2))

---

## References

- [API Gateway Pattern - Microsoft](https://docs.microsoft.com/en-us/azure/architecture/microservices/design/gateway)
- [Kong Gateway Documentation](https://docs.konghq.com/)
- [Envoy Proxy](https://www.envoyproxy.io/docs/envoy/latest/)
- [MCP Specification - Transports](https://modelcontextprotocol.io/docs/concepts/transports)

---

**Next Notebook:** [10_Enterprise_MCP_Patterns.ipynb](./10_Enterprise_MCP_Patterns.ipynb)